In [1]:
import os
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.core.agent.react import ReActAgent
from llama_index.core.workflow import (
    step,
    Context,
    Workflow,
    Event,
    StartEvent,
    StopEvent
)
from llama_index.llms.openai import OpenAI
from llama_index.core.postprocessor.rankGPT_rerank import RankGPTRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.utils.workflow import draw_all_possible_flows


In [2]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
class JudgeEvent(Event):
    query: str

class BadQueryEvent(Event):
    query: str

class NaiveRAGEvent(Event):
    query: str

class HighTopKEvent(Event):
    query: str

class RerankEvent(Event):
    query: str

class ResponseEvent(Event):
    query: str
    response: str

class SummarizeEvent(Event):
    query: str
    response: str

In [4]:
class ComplicatedWorkflow(Workflow):

    def load_or_create_index(self, directory_path, persist_dir):
        if os.path.exists(persist_dir):
            print("Loading existing index...")
            storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
            index = load_index_from_storage(storage_context)
        else:
            print("Creating new index...")
            documents = SimpleDirectoryReader(directory_path, recursive=True).load_data()
            index = VectorStoreIndex.from_documents(documents)
            index.storage_context.persist(persist_dir=persist_dir)
        return index

    @step(pass_context=True)
    async def judge_query(self, ctx: Context, ev: StartEvent | JudgeEvent ) -> BadQueryEvent | NaiveRAGEvent | HighTopKEvent | RerankEvent:

        # initialize
        if not hasattr(ctx.data, "llm"):
            ctx.data["llm"] = OpenAI(model="gpt-4o",temperature=0.1)
            ctx.data["index"] = self.load_or_create_index(
                "data/get-started/",
                "storage"
            )
            # we use a chat engine so it remembers previous interactions
            ctx.data["judge"] = SimpleChatEngine.from_defaults()

        response = ctx.data["judge"].chat(f"""
            Given a user query, determine if this is likely to yield good results from a RAG system as-is. If it's good, return 'good', if it's bad, return 'bad'.
            Good queries use a lot of relevant keywords and are detailed. Bad queries are vague or ambiguous.

            Here is the query: {ev.query}
            """)
        if response == "bad":
            # try again
            return BadQueryEvent(query=ev.query)
        else:
            # send query to all 3 strategies
            self.send_event(NaiveRAGEvent(query=ev.query))
            self.send_event(HighTopKEvent(query=ev.query))
            self.send_event(RerankEvent(query=ev.query))

    @step(pass_context=True)
    async def improve_query(self, ctx: Context, ev: BadQueryEvent) -> JudgeEvent:
        response = ctx.data["llm"].complete(f"""
            This is a query to a RAG system: {ev.query}

            The query is bad because it is too vague. Please provide a more detailed query that includes specific keywords and removes any ambiguity.
        """)
        return JudgeEvent(query=str(response))

    @step(pass_context=True)
    async def naive_rag(self, ctx: Context, ev: NaiveRAGEvent) -> ResponseEvent:
        index = ctx.data["index"]
        engine = index.as_query_engine(similarity_top_k=5)
        response = engine.query(ev.query)
        print("Naive response:", response)
        return ResponseEvent(query=ev.query, source="Naive", response=str(response))

    @step(pass_context=True)
    async def high_top_k(self, ctx: Context, ev: HighTopKEvent) -> ResponseEvent:
        index = ctx.data["index"]
        engine = index.as_query_engine(similarity_top_k=20)
        response = engine.query(ev.query)
        print("High top k response:", response)
        return ResponseEvent(query=ev.query, source="High top k", response=str(response))

    @step(pass_context=True)
    async def rerank(self, ctx: Context, ev: RerankEvent) -> ResponseEvent:
        index = ctx.data["index"]
        reranker = RankGPTRerank(
            top_n=5,
            llm=ctx.data["llm"]
        )
        retriever = index.as_retriever(similarity_top_k=20)
        engine = RetrieverQueryEngine.from_args(
            retriever=retriever,
            node_postprocessors=[reranker],
        )
        response = engine.query(ev.query)
        print("Reranker response:", response)
        return ResponseEvent(query=ev.query, source="Reranker", response=str(response))

    @step(pass_context=True)
    async def judge(self, ctx: Context, ev: ResponseEvent) -> StopEvent:
        ready = ctx.collect_events(ev, [ResponseEvent]*3)
        if ready is None:
            return None

        response = ctx.data["judge"].chat(f"""
            A user has provided a query and 3 different strategies have been used
            to try to answer the query. Your job is to decide which strategy best
            answered the query. The query was: {ev.query}

            Response 1 ({ready[0].source}): {ready[0].response}
            Response 2 ({ready[1].source}): {ready[1].response}
            Response 3 ({ready[2].source}): {ready[2].response}

            Please provide the number of the best response (1, 2, or 3).
            Just provide the number, with no other text or preamble.
        """)

        best_response = int(str(response))
        print(f"Best response was number {best_response}, which was from {ready[best_response-1].source}")
        return StopEvent(result=str(ready[best_response-1].response))

In [5]:
draw_all_possible_flows(ComplicatedWorkflow,filename="complicated_workflow.html")

complicated_workflow.html


In [6]:
c = ComplicatedWorkflow(timeout=120, verbose=True)
result = await c.run(
    query="How to install llama_index on cloud?"
)
print(result)

Running step judge_query
Loading existing index...
Step judge_query produced no event
Running step naive_rag
Naive response: To install llama_index on cloud, you would typically need to follow the specific installation instructions provided by the llama_index software documentation or the cloud platform's guidelines for installing third-party software. This may involve using package managers, running specific commands, configuring settings, and ensuring compatibility with the cloud environment.
Step naive_rag produced event ResponseEvent
Running step rerank
Reranker response: To install llama_index on cloud, you would typically need to follow the specific installation instructions provided by the llama_index software documentation or the cloud platform's documentation. This usually involves steps such as setting up the necessary environment, installing any dependencies, configuring the software, and deploying it on the cloud platform according to their guidelines.
Step rerank produced 